In [1]:
import os
import sys
import yaml
import json

with open("config.yaml", 'r') as yml_file:
    cfg = yaml.safe_load(yml_file)
    
#Get data path from config 
data_path = cfg['brian_laptop']['data_path']
mb_div = cfg['contants']['mb_div']

#Get dyype for meta file 
meta_data_file = os.path.join(data_path, cfg['data']['metadata'])
with open(meta_data_file) as f:
    meta_data = json.load(f)

In [2]:
import datatable as dt
print(dt.__version__)

0.9.0


## Bigger Data

12 million rows, 32 columns

In [3]:
%%time

nyc_taxi_fare_file = os.path.join(data_path, cfg['data']['kaggle_nyc'])
nyc_taxi_fare = dt.fread(nyc_taxi_fare_file)

CPU times: user 58.8 ms, sys: 4.9 ms, total: 63.7 ms
Wall time: 17.7 ms


In [4]:
print("data table memory(MB)", round(sys.getsizeof(nyc_taxi_fare)/mb_div,3))
print(nyc_taxi_fare.shape)

data table memory(MB) 0.823
(10000, 8)


In [5]:
nyc_taxi_fare[:5,:]

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,24:57.0,15.3,2011-05-28 14:24:57 UTC,−74.004,40.7073,−73.9873,40.7566,1
1,04:00.0,10.5,2010-06-13 00:04:00 UTC,−73.9853,40.7274,−73.9832,40.7568,1
2,40:23.0,15.7,2012-06-01 21:40:23 UTC,−73.9919,40.764,−73.9946,40.7184,1
3,56:00.0,5,2014-03-03 21:56:00 UTC,−73.9825,40.6921,−73.969,40.6941,2
4,21:10.0,6.9,2009-05-07 07:21:10 UTC,−73.9972,40.7493,−73.9821,40.7363,4


In [30]:
#f is a frame proxy used to refer to the frame that is currently being operated on. 
nyc_taxi_fare[:, (dt.count(dt.f.fare_amount), 
                  dt.mean(dt.f.fare_amount),
                  dt.sd(dt.f.fare_amount),
                  dt.min(dt.f.fare_amount),
                  dt.median(dt.f.fare_amount),
                  dt.max(dt.f.fare_amount)),
              dt.by("passenger_count")]

,passenger_count,C0,C1,C2,C3,C4,C5
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,24,8.19167,4.09538,2.5,7.3,17.7
1,1,6973,11.4051,9.73905,2.5,8.5,162.75
2,2,1445,12.242,13.5485,2.5,8.5,331.29
3,3,467,11.2602,10.0857,2.5,8.5,77.25
4,4,208,10.2295,8.60077,3.3,7.7,87.5
5,5,673,11.6702,9.49931,2.5,8.5,63.7
6,6,210,10.4765,7.07156,2.5,8.5,52


In [42]:
del(nyc_taxi_fare)

## PYDATATABLE INTERESTING BITS ##